In [6]:
%%capture
!uv pip install polars

In [7]:
import polars as pl

In [8]:
df = pl.read_csv("../.data/chess_games_2025-01-15.csv", null_values=["None"])

In [9]:
df.head()

Date,Result,WhiteElo,BlackElo,PGN
str,str,i64,i64,str
"""2000.03.14""","""1-0""",2851,null,"""1.d4 d5 2.c4 e6 3.Nc3 Nf6 4.cx…"
"""2000.03.14""","""1-0""",2851,null,"""1.e4 d5 2.exd5 Qxd5 3.Nc3 Qa5 …"
"""1999.11.20""","""1-0""",2851,null,"""1.e4 e5 2.Nf3 Nc6 3.Bc4 Bc5 4.…"
"""1999.11.20""","""1-0""",2851,null,"""1.e4 d5 2.exd5 Qxd5 3.Nc3 Qa5 …"
"""2000.02.20""","""1/2-1/2""",2851,2633,"""1.e4 e5 2.Nf3 Nc6 3.Bb5 a6 4.B…"


In [10]:
test_game = df.select("PGN")[0].item()

In [11]:
tokens = test_game.encode("utf-8")
token_ids = list(map(int, tokens))

print(test_game)
print("length: ", len(test_game))
print(token_ids)
print("length:", len(token_ids))

1.d4 d5 2.c4 e6 3.Nc3 Nf6 4.cxd5 exd5 5.Bg5 Be7 6.e3 Ne4 7.Bxe7 Nxc3 8.Bxd8 Nxd1 9.Bxc7 Nxb2 10.Rb1 Nc4 11.Bxc4 dxc4 12.Ne2 O-O 13.Nc3 b6 14.d5 Na6 15.Bd6 Rd8 16.Ba3 Bb7 17.e4 f6 18.Ke2 Nc7 19.Rhd1 Ba6 20.Ke3 Kf7 21.g4 g5 22.h4 h6 23.Rh1 Re8 24.f3 Bb7 25.hxg5 fxg5 26.d6 Nd5+ 27.Nxd5 Bxd5 28.Rxh6 c3 29.d7 Re6 30.Rh7+ Kg8 31.Rbh1 Bc6 32.Rh8+ Kf7 33.Rxa8 Bxd7 34.Rh7+ 
length:  367
[49, 46, 100, 52, 32, 100, 53, 32, 50, 46, 99, 52, 32, 101, 54, 32, 51, 46, 78, 99, 51, 32, 78, 102, 54, 32, 52, 46, 99, 120, 100, 53, 32, 101, 120, 100, 53, 32, 53, 46, 66, 103, 53, 32, 66, 101, 55, 32, 54, 46, 101, 51, 32, 78, 101, 52, 32, 55, 46, 66, 120, 101, 55, 32, 78, 120, 99, 51, 32, 56, 46, 66, 120, 100, 56, 32, 78, 120, 100, 49, 32, 57, 46, 66, 120, 99, 55, 32, 78, 120, 98, 50, 32, 49, 48, 46, 82, 98, 49, 32, 78, 99, 52, 32, 49, 49, 46, 66, 120, 99, 52, 32, 100, 120, 99, 52, 32, 49, 50, 46, 78, 101, 50, 32, 79, 45, 79, 32, 49, 51, 46, 78, 99, 51, 32, 98, 54, 32, 49, 52, 46, 100, 53, 32, 78, 97, 54, 32,

In [12]:
# get the most common pair of tokens
def find_token_pairs(ids):
    counts = {}

    # get pairs of tokens
    for pair in zip(ids, ids[1:]):
        if pair not in counts:
            counts[pair] = 1
        else:
            counts[pair] += 1

    return counts

In [13]:
data_start = "<|startgame|>"
data_end = "<|endgame|>"

data_start_tokens = data_start.encode("utf-8")
data_end_tokens = data_end.encode("utf-8")

data_start_token_ids = list(map(int, data_start_tokens))
data_end_token_ids = list(map(int, data_end_tokens))

print(data_start_token_ids)
print(data_end_token_ids)

tokenizer_special_tokens = {
    "startgame": data_start,
    "endgame": data_end,
}

[60, 124, 115, 116, 97, 114, 116, 103, 97, 109, 101, 124, 62]
[60, 124, 101, 110, 100, 103, 97, 109, 101, 124, 62]


In [14]:
token_pairs = find_token_pairs(token_ids)

# print(tokenPairs)
print(sorted(((v, k) for k, v in token_pairs.items()), reverse=True))

[(12, (54, 32)), (11, (32, 50)), (10, (55, 32)), (10, (53, 32)), (10, (32, 49)), (9, (52, 32)), (9, (46, 82)), (9, (32, 78)), (8, (51, 32)), (7, (120, 100)), (7, (100, 53)), (7, (46, 66)), (7, (32, 66)), (6, (66, 120)), (6, (32, 51)), (5, (82, 104)), (5, (56, 32)), (5, (49, 32)), (4, (120, 99)), (4, (103, 53)), (4, (99, 52)), (4, (99, 51)), (4, (78, 120)), (4, (78, 99)), (4, (52, 46)), (4, (51, 46)), (4, (50, 46)), (4, (49, 46)), (4, (46, 100)), (4, (46, 78)), (4, (43, 32)), (3, (57, 46)), (3, (56, 46)), (3, (55, 46)), (3, (54, 46)), (3, (53, 46)), (3, (50, 32)), (3, (48, 46)), (3, (32, 82)), (3, (32, 75)), (2, (120, 103)), (2, (104, 55)), (2, (104, 54)), (2, (104, 49)), (2, (102, 55)), (2, (102, 54)), (2, (101, 55)), (2, (101, 54)), (2, (101, 52)), (2, (101, 51)), (2, (101, 50)), (2, (100, 56)), (2, (100, 55)), (2, (100, 54)), (2, (100, 49)), (2, (99, 55)), (2, (98, 55)), (2, (97, 54)), (2, (82, 120)), (2, (82, 101)), (2, (82, 98)), (2, (78, 101)), (2, (75, 102)), (2, (75, 101)), (2, 

In [15]:
top_pair = max(token_pairs, key=token_pairs.get)
top_pair

(54, 32)

In [16]:
def merge_pairs(ids, pair, idx):
    new_ids = []

    i = 0

    while i < len(ids):
        if i < len(ids) - 1 and ids[i] == pair[0] and ids[i + 1] == pair[1]:
            new_ids.append(idx)
            i += 2
        else:
            new_ids.append(ids[i])
            i += 1

        return new_ids

In [17]:
tokens2 = merge_pairs(token_ids, top_pair, 256)

print(tokens2)
print("length:", len(tokens2))

[49]
length: 1


In [76]:
sample = df.select("PGN").sample(n=500)

In [78]:
text = ""

for game in sample.iter_rows():
    if game[0]:
        text += game[0].strip() + "\n"

In [79]:
text_tokens = text.encode("utf-8")
text_tokens = list(map(int, text_tokens))

In [80]:
def get_stats(ids):
    counts = {}
    for pair in zip(ids, ids[1:]):
        counts[pair] = counts.get(pair, 0) + 1
    return counts


def merge_pairs(ids, pair, idx):
    new_ids = []
    i = 0
    while i < len(ids):
        if i < len(ids) - 1 and ids[i] == pair[0] and ids[i + 1] == pair[1]:
            new_ids.append(idx)
            i += 2
        else:
            new_ids.append(ids[i])
            i += 1
    return new_ids


vocab_size = 512
number_of_merges = vocab_size - 256
ids = list(text_tokens)

merges = {}

for i in range(number_of_merges):
    stats = get_stats(ids)
    pair = max(stats, key=stats.get)
    idx = 256 + i
    print(f"merge {pair} into a new token {idx}")
    ids = merge_pairs(ids, pair, idx)
    merges[pair] = idx

merge (52, 32) into a new token 256
merge (53, 32) into a new token 257
merge (54, 32) into a new token 258
merge (51, 32) into a new token 259
merge (55, 32) into a new token 260
merge (46, 78) into a new token 261
merge (46, 82) into a new token 262
merge (50, 32) into a new token 263
merge (46, 66) into a new token 264
merge (56, 32) into a new token 265
merge (46, 81) into a new token 266
merge (49, 32) into a new token 267
merge (120, 100) into a new token 268
merge (43, 32) into a new token 269
merge (46, 75) into a new token 270
merge (120, 101) into a new token 271
merge (120, 99) into a new token 272
merge (120, 102) into a new token 273
merge (257, 49) into a new token 274
merge (257, 50) into a new token 275
merge (258, 49) into a new token 276
merge (258, 50) into a new token 277
merge (79, 45) into a new token 278
merge (102, 259) into a new token 279
merge (46, 101) into a new token 280
merge (278, 79) into a new token 281
merge (281, 32) into a new token 282
merge (258, 

In [81]:
print("text length:", len(text))
print("tokens length:", len(text_tokens))
print("ids length:", len(ids))
print(f"compression ratio: {len(text_tokens) / len(ids):.2f}")

text length: 216721
tokens length: 216721
ids length: 93456
compression ratio: 2.32


In [82]:
vocab = {idx: bytes([idx]) for idx in range(256)}

for (p0, p1), idx in merges.items():
    vocab[idx] = vocab[p0] + vocab[p1]


def decode(ids: list[int]) -> str:
    tokens = b"".join(vocab[idx] for idx in ids)

    text = tokens.decode("utf-8", errors="replace")

    return text


decode(ids)

'1.e4 c5 2.Nf3 Nc6 3.d4 cxd4 4.Nxd4 Nf6 5.Nc3 e5 6.Ndb5 d6 7.Nd5 Nxd5 8.exd5 Ne7 9.c3 Ng6 10.Qa4 Bd7 11.Qc4 Rc8 12.Qb4 Qb6 13.Be3 Qd8 14.Nxd6+ Bxd6 15.Qxd6 Qa5 16.Be2 Rxc3 17.Bd2 Qc7 18.Qxc7 Rxc7 19.O-O Rc2 20.Rfd1 Rxb2 21.a4 O-O 22.Bf3 Rc8 23.Be3 Rcc2 24.Be4 Rc4 25.Bd3 Rxa4 26.Rxa4 Bxa4 27.Rc1 Kf8 28.h3 Ke7 29.Bxa7 Nf4 30.Bc5+ Kf6 31.Bf1 Nxd5 32.Ra1 Bc2 33.Bc4 Nf4 34.Re1 Nd3 35.Bxd3 Bxd3 36.Bd6 Rb1 37.Bxe5+ Kg6 38.Rxb1 Bxb1 39.Kf1 b5 40.Bc3 f5 41.Ke2 Kf7 42.Ke3 Be4 43.f3 Bd5 44.Kd4 Bc4 45.Ke5 Bf1 46.g4 g6 47.gxf5 gxf5 48.Kxf5 Bxh3+ 49.Kg5 Ke6 50.Kh6 Bf5 51.Kg5 Bg6 52.f4 Be4 53.Bb4 Bd3 54.Ba3 Be4 55.Bb4 Bg6 56.Bc3 Bd3\n1.d4 g6 2.c4 Bg7 3.Nc3 d6 4.e4 Nc6 5.Be3 e5 6.d5 Nce7 7.b4 f5 8.f3 a5 9.a3 Nf6 10.Bd3 O-O 11.c5 fxe4 12.fxe4 Ng4 13.Ke2 Nxe3 14.Kxe3 Nxd5+ 15.Nxd5 Qg5+ 16.Ke2 Qxg2+ 17.Ke1 Qf2#\n1.d4 Nf6 2.c4 g6 3.Nf3 Bg7 4.Nc3 O-O 5.Bg5 c5 6.e3 cxd4 7.exd4 h6 8.Bh4 d6 9.Qd2 Bg4 10.Be2 Nfd7 11.Rd1 Nc6 12.Ng1 Bxe2 13.Ngxe2 Re8 14.O-O Qa5 15.f4 Rac8 16.Bf2 a6 17.d5 Ncb8 18.b3 b5 19.Ne4 Qxd

In [83]:
def encode(text: str) -> list[int]:
    tokens = list(text.encode("utf-8"))

    while len(tokens) >= 2:
        stats = get_stats(tokens)

        pair = min(stats, key=lambda p: merges.get(p, float("inf")))

        if pair not in merges:
            break

        idx = merges[pair]

        tokens = merge_pairs(tokens, pair, idx)

    return tokens


print(encode("1.d4 d5 2.Nf3 Bf5"))

[49, 284, 352, 413, 66, 102, 53]


In [84]:
print(decode(encode("1.d4 d5 2.Nf3 Bf5")))

1.d4 d5 2.Nf3 Bf5


In [85]:
new_sample = df.select("PGN").sample(n=1)[0].item().strip()

print(new_sample == decode(encode(new_sample)))

True


In [86]:
import re

# ignore `1.`, ` 2.`, ` `, etc. and get the actual moves as separate entries
tokenizer_pattern = re.compile(
    r""" ?\d+\.|\. ?| ?[-\w]+|[#+]|\s+""", flags=re.IGNORECASE
)

print(re.findall(tokenizer_pattern, new_sample))

['1.', 'd4', ' Nf6', ' 2.', 'Bg5', ' e6', ' 3.', 'Nd2', ' d5', ' 4.', 'e3', ' Be7', ' 5.', 'Bd3', ' O-O', ' 6.', 'f4', ' h6', ' 7.', 'Bh4', ' Nbd7', ' 8.', 'Ngf3', ' Ng4', ' 9.', 'Bxe7', ' Qxe7', ' 10.', 'Qe2', ' f5', ' 11.', 'h3', ' Ngf6', ' 12.', 'O-O', ' Ne4', ' 13.', 'Bxe4', ' fxe4', ' 14.', 'Ne5', ' Nxe5', ' 15.', 'fxe5', ' Bd7', ' 16.', 'Qg4', ' Qb4', ' 17.', 'Nb3', ' b6', ' 18.', 'Rad1', ' Rxf1', '+', ' 19.', 'Rxf1', ' a5', ' 20.', 'c3', ' Qe7', ' 21.', 'Nc1', ' Rf8', ' 22.', 'Rxf8', '+', ' Kxf8', ' 23.', 'Ne2', ' g5', ' 24.', 'Ng3', ' Be8', ' 25.', 'Qe2', ' b5', ' 26.', 'Nf1', ' a4', ' 27.', 'Nd2', ' Qf7', ' 28.', 'Nb1', ' Qh5', ' 29.', 'Qf2', '+', ' Qf7', ' 30.', 'Qe1', ' Qf5', ' 31.', 'Na3', ' g4', ' 32.', 'Qh4', ' h5', ' 33.', 'hxg4', ' hxg4', ' 34.', 'Kh2', ' c6', ' 35.', 'Nc2', ' Qh5', ' 36.', 'Qxh5', ' Bxh5', ' 37.', 'Nb4', ' Be8', ' 38.', 'Kg3', ' Kg7', ' 39.', 'Kxg4', ' Kg6', ' 40.', 'Na6', ' Bd7', ' 41.', 'Nb8', ' Be8', ' 42.', 'Kh4', ' Kh6', ' 43.', 'g4', ' Kg6', ' 44

In [87]:
def encode_with_regex_splits(text: str) -> list[int]:
    bpe_tokens = []

    for tokens in re.findall(tokenizer_pattern, text):
        token_ids = list(tokens.encode("utf-8"))

        while len(token_ids) >= 2:
            stats = get_stats(token_ids)

            pair = min(stats, key=lambda p: merges.get(p, float("inf")))

            if pair not in merges:
                break

            idx = merges[pair]

            token_ids = merge_pairs(token_ids, pair, idx)

        bpe_tokens.extend(token_ids)

    return bpe_tokens


print(encode_with_regex_splits("1.d4 d5 2.Nf3 Bf5"))

encoded = encode_with_regex_splits("1.d4 d5 2.Nf3 Bf5")

for idx in encoded:
    print(vocab[idx].decode("utf-8"))

[49, 46, 100, 52, 32, 100, 53, 32, 50, 46, 353, 51, 32, 66, 102, 53]
1
.
d
4
 
d
5
 
2
.
Nf
3
 
B
f
5


In [94]:
text_chunks = re.findall(tokenizer_pattern, text)


def get_stats(ids: list, counts: dict = None) -> dict:
    counts = {} if counts is None else counts

    for pair in zip(ids, ids[1:]):
        counts[pair] = counts.get(pair, 0) + 1

    return counts


def merge_pairs(ids, pair, idx):
    new_ids = []
    i = 0
    while i < len(ids):
        if i < len(ids) - 1 and ids[i] == pair[0] and ids[i + 1] == pair[1]:
            new_ids.append(idx)
            i += 2
        else:
            new_ids.append(ids[i])
            i += 1
    return new_ids


vocab_size = 3072
number_of_merges = vocab_size - 256
ids = [list(ch.encode("utf-8")) for ch in text_chunks]

merges = {}
vocab = {idx: bytes([idx]) for idx in range(256)}

for i in range(number_of_merges):
    stats = {}

    for chunk_ids in ids:
        get_stats(chunk_ids, stats)

    if not stats or len(stats) < 1:
        break

    pair = max(stats, key=stats.get)

    idx = 256 + i

    ids = [merge_pairs(chunk_ids, pair, idx) for chunk_ids in ids]

    merges[pair] = idx

    vocab[idx] = vocab[pair[0]] + vocab[pair[1]]

    print(
        f"merge{i+1}/{number_of_merges} {pair} -> {idx} (\"{vocab[idx].decode('utf-8')}\": {stats[pair]} occurences)"
    )

merge1/2816 (32, 49) -> 256 (" 1": 4731 occurences)
merge2/2816 (32, 50) -> 257 (" 2": 4630 occurences)
merge3/2816 (32, 51) -> 258 (" 3": 3544 occurences)
merge4/2816 (32, 78) -> 259 (" N": 3359 occurences)
merge5/2816 (32, 82) -> 260 (" R": 3139 occurences)
merge6/2816 (32, 66) -> 261 (" B": 3137 occurences)
merge7/2816 (32, 52) -> 262 (" 4": 2371 occurences)
merge8/2816 (32, 81) -> 263 (" Q": 2320 occurences)
merge9/2816 (49, 46) -> 264 ("1.": 2208 occurences)
merge10/2816 (53, 46) -> 265 ("5.": 2012 occurences)
merge11/2816 (120, 100) -> 266 ("xd": 2007 occurences)
merge12/2816 (54, 46) -> 267 ("6.": 1960 occurences)
merge13/2816 (55, 46) -> 268 ("7.": 1911 occurences)
merge14/2816 (32, 75) -> 269 (" K": 1899 occurences)
merge15/2816 (56, 46) -> 270 ("8.": 1858 occurences)
merge16/2816 (57, 46) -> 271 ("9.": 1810 occurences)
merge17/2816 (48, 46) -> 272 ("0.": 1757 occurences)
merge18/2816 (50, 46) -> 273 ("2.": 1664 occurences)
merge19/2816 (51, 46) -> 274 ("3.": 1622 occurences)


In [75]:
def encode_with_regex_splits(text: str) -> list[int]:
    bpe_tokens = []

    for tokens in re.findall(tokenizer_pattern, text):
        token_ids = list(tokens.encode("utf-8"))

        while len(token_ids) >= 2:
            stats = get_stats(token_ids)

            pair = min(stats, key=lambda p: merges.get(p, float("inf")))

            if pair not in merges:
                break

            idx = merges[pair]

            token_ids = merge_pairs(token_ids, pair, idx)

        bpe_tokens.extend(token_ids)

    return bpe_tokens


encoded = encode_with_regex_splits("1.d4 d5 2.Nf3 Bf5")
print(encoded)

for idx in encoded:
    print(f'"{vocab[idx].decode("utf-8")}"', end=" ")

print("\n---\n")

encoded2 = encode_with_regex_splits(new_sample)
print(encoded2)

for idx in encoded2:
    print(f'"{vocab[idx].decode("utf-8")}"', end=" ")

[263, 282, 374, 314, 311, 489]
"1." "d4" " d5" " 2." "Nf3" " Bf5" 
---

[263, 291, 402, 314, 282, 374, 315, 498, 588, 316, 400, 361, 317, 288, 406, 318, 440, 395, 319, 311, 489, 320, 770, 651, 321, 281, 383, 323, 451, 389, 324, 446, 302, 325, 807, 515, 326, 927, 275, 490, 327, 300, 646, 328, 532, 669, 332, 453, 877, 431, 333, 279, 302, 334, 603, 559, 337, 1070, 568, 370, 338, 81, 443, 43, 545, 343, 353, 608, 346, 322, 43, 767, 347, 921, 259, 359, 349, 1010, 777, 352, 350, 689, 830, 354, 78, 370, 43, 568, 322, 357, 418, 277, 1020, 43, 360, 547, 621, 43, 363, 835, 259, 359, 43, 366, 596, 413, 367, 43]
"1." "e4" " c6" " 2." "d4" " d5" " 3." "exd5" " cxd5" " 4." "Bd3" " Nc6" " 5." "c3" " Qc7" " 6." "Bg5" " g6" " 7." "Nf3" " Bf5" " 8." "Bxf5" " gxf5" " 9." "O-O" " e6" " 10." "Qe2" " Be7" " 11." "Re1" " Nf6" " 12." "Na3" " Ne4" " 13." "Bxe7" " K" "xe7" " 14." "c4" " Rad8" " 15." "Rad1" " Nb4" " 16." "Ne5" " Rh" "g8" " 17." "f3" " Nf6" " 18." "Nb5" " Qb8" " 19." "Nxf7" " Kx" "f7" " 20." "Q" "